In [1]:
%matplotlib widget

import os  
import sys  
sys.path.insert(0, os.getenv('HOME')+'/pycode/MscThesis/')
import pandas as pd
from amftrack.util import get_dates_datetime, get_dirname, get_plate_number, get_postion_number

import ast
from amftrack.plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime
from amftrack.pipeline.functions.node_id import orient
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from amftrack.pipeline.functions.extract_graph import from_sparse_to_graph, generate_nx_graph, sparse_to_doc
from skimage.feature import hessian_matrix_det
from amftrack.pipeline.functions.experiment_class_surf import Experiment
from amftrack.pipeline.paths.directory import run_parallel, find_state, directory_scratch, directory_project, path_code

from amftrack.notebooks.analysis.data_info import *
import matplotlib.patches as mpatches
from statsmodels.stats import weightstats as stests


In [2]:
window=800
results={}
for treatment in treatments.keys():
    insts = treatments[treatment]
    for inst in insts:
        results[inst] = pickle.load(open(f'{path_code}/MscThesis/Results/straight_{window}_{inst}.pick', "rb"))

In [3]:
column_names = ["plate","inst", "treatment", "angle", "curvature","density","growth","speed","straightness","t","hyph","total_network"]
infos = pd.DataFrame(columns=column_names)
for treatment in treatments.keys():
    insts = treatments[treatment]
    for inst in insts:
        angles, curvatures, densities,growths,speeds,tortuosities,ts,hyphs,total_lengths = results[inst]
        for i,angle in enumerate(angles):
            new_line = pd.DataFrame(
                {   "plate": [plate_number[inst]],
                    "inst": [inst],
                    "treatment": [treatment],
                    "angle": [angle],
                    "curvature": [curvatures[i]],
                    "density": [densities[i]],
                    "growth": [growths[i]],
                    "speed": [speeds[i]],
                    "straightness": [tortuosities[i]],
                     "t": [ts[i]],
                     "hyph": [hyphs[i]],
                     "total_network" : [total_lengths[i]],
                }
            )  # index 0 for
            # mothers need to be modified to resolve multi mother issue
            infos = infos.append(new_line, ignore_index=True)

In [18]:
speed[:10]

array([1608.25035625,  267.91909801,  359.76637183, -116.37934115,
         66.06112056, -625.74896763,  167.38691282, -112.77395878,
       -171.45343585,  -61.35774973])

In [19]:
x_values = infos['growth'].values
speed= x_values[:-1]-x_values[1:]
speed_complete= np.insert(speed,0,np.nan)
infos['speed2'] = speed_complete

In [4]:
corrected = infos.loc[(infos["straightness"] <= 1)&(infos["plate"] !=435)& (infos["speed"] >=25)& (infos["speed"] <400)]

In [5]:
plt.close('all')
plate_list = [94]
plates = corrected.loc[corrected['plate'].isin(plate_list)]

In [6]:
total_growth=[]
for t in set(plate_94['t'].values):
    select = plate_94.loc[plate_94['t']==t]
    total_growth.append(np.sum(select['speed']))
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(9, 4))
cum_growth = np.cumsum(total_growth)
ax.scatter(list(set(list(plate_94['t'].values))),np.array(cum_growth)/cum_growth[0],label='observed speeds')

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/__init__.py:880: MatplotlibDeprecationWarning: 
nbagg.transparent
  version, key, obj_type="rcparam", alternative=alt_key)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [39]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(9, 4))
cum_growth = np.cumsum(total_growth)
ax.scatter(list(set(list(plate_94['t'].values))),np.array(total_growth)/total_growth[0],label='observed speeds')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
from scipy.optimize import curve_fit
from sklearn.linear_model import LinearRegression

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(9, 4))

def func(x,a,b):
    return(a*np.exp(b*x))
times = list(set(list(plate_94['t'].values)))
X = np.array(times).reshape(-1,1)
y = np.log(np.array(total_growth))
reg = LinearRegression().fit(X, y)
ax.scatter(times,np.array(total_growth)/total_growth[0])
ax.plot(times,np.exp(times*reg.coef_),label='length rh')

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/__init__.py:880: MatplotlibDeprecationWarning: 
nbagg.transparent
  version, key, obj_type="rcparam", alternative=alt_key)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [41]:
total_growth

[274.4182939842498,
 264.75066857923247,
 130.14008759834883,
 314.0507474032412,
 113.7477630475859,
 1161.622784505229,
 1745.0546022253006,
 2726.9218066428225,
 2871.778197123677,
 4383.75894167157,
 4214.542457713185,
 3085.5196044227305,
 3814.118401873534,
 3850.501563058079,
 2799.2820758732155,
 644.4555313848887,
 1955.058800564103,
 488.4187953798312,
 5437.132269415376,
 6029.556668562571,
 309.74440034631823,
 2827.3429474183235,
 7237.124025346528,
 6157.3855798877885]

In [27]:
reg.coef_*24

array([0.92487663])

In [33]:
popt0

array([ 7.52080445e+04,  2.74393313e-02, -8.39831995e+04])

In [9]:
def func(x,a,b,c):
    return(a*np.exp(b*x)+c)
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(9, 4))
popt0, pcov = curve_fit(func, times, cum_growth,bounds = ([0,0,-np.inf],3*[np.inf]),p0=[1,1,0])
ax.scatter(times,(cum_growth-popt0[2])/popt0[0])
coef = popt0[-2]
ax.plot(times,np.exp(np.array(times)*coef),label='total_length')


/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/__init__.py:880: MatplotlibDeprecationWarning: 
nbagg.transparent
  version, key, obj_type="rcparam", alternative=alt_key)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
coef*24

0.29643288601140516

In [7]:
plt.close("all")
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(9, 4))
ax.scatter(plate_94['t'],plate_94['speed'])

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/__init__.py:880: MatplotlibDeprecationWarning: 
nbagg.transparent
  version, key, obj_type="rcparam", alternative=alt_key)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [63]:
baits_sort = plate_94.sort_values("t")
N=80
moving_av = baits_sort.rolling(N).mean()
moving_std = baits_sort.rolling(N).std()

fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.set_xlabel('time(t)')
ax.set_ylabel('speed($\mu m. h^{-1}$)')
xlim = 40
# ax.set_xlim(0,60)
ax.set_ylim(0,400)
# slope, intercept, r_value, p_value, std_err = stats.linregress(densities_sort,np.abs(curvatures_sort))
x= range(-xlim,xlim)

ax.scatter(plate_94["t"],plate_94['speed'],color='yellow',label='observed speeds')
ax.plot(moving_av["t"],moving_av['speed'],color='green',label = 'moving average')
ax.plot(moving_av["t"],(moving_av['speed']+moving_std['speed']/np.sqrt(N)),color='red',label = 'moving average')
ax.plot(moving_av["t"],(moving_av['speed']-moving_std['speed']/np.sqrt(N)),color='red',label = 'moving average')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [64]:
baits_sort = plate_94.sort_values("total_network")
N=80
moving_av = baits_sort.rolling(N).mean()
moving_std = baits_sort.rolling(N).std()

fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.set_xlabel('total network($\mu m$)')
ax.set_ylabel('speed($\mu m. h^{-1}$)')
xlim = 40
# ax.set_xlim(0,60)
ax.set_ylim(0,400)
# slope, intercept, r_value, p_value, std_err = stats.linregress(densities_sort,np.abs(curvatures_sort))
x= range(-xlim,xlim)

ax.scatter(plate_94["total_network"],plate_94['speed'],color='yellow',label='observed speeds')
ax.plot(moving_av["total_network"],moving_av['speed'],color='green',label = 'moving average')
ax.plot(moving_av["total_network"],(moving_av['speed']+moving_std['speed']/np.sqrt(N)),color='red',label = 'moving average')
ax.plot(moving_av["total_network"],(moving_av['speed']-moving_std['speed']/np.sqrt(N)),color='red',label = 'moving average')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [47]:
treatment_25 = corrected.loc[corrected['treatment']=='baits']

In [48]:
set(treatment_25['plate'].values)

{382, 395, 399, 405, 409, 416, 419, 420, 423, 424, 425, 433, 436}

In [61]:
plt.close('all')
baits_sort = treatment_25.sort_values("total_network")
N=200
moving_av = baits_sort.rolling(N).mean()
moving_std = baits_sort.rolling(N).std()

fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.set_xlabel('total network($\mu m$)')
ax.set_ylabel('speed($\mu m. h^{-1}$)')
xlim = 40
# ax.set_xlim(0,60)
ax.set_ylim(0,400)
# slope, intercept, r_value, p_value, std_err = stats.linregress(densities_sort,np.abs(curvatures_sort))
x= range(-xlim,xlim)
for plate in set(treatment_25['plate'].values):
    select = treatment_25.loc[treatment_25['plate']==plate]
    ax.scatter(select["total_network"],select['speed'],label='observed speeds')
ax.plot(moving_av["total_network"],moving_av['speed'],color='green',label = 'moving average')
ax.plot(moving_av["total_network"],(moving_av['speed']+moving_std['speed']/np.sqrt(N)),color='red',label = 'moving average')
ax.plot(moving_av["total_network"],(moving_av['speed']-moving_std['speed']/np.sqrt(N)),color='red',label = 'moving average')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
plt.close('all')
baits_sort = treatment_25.sort_values("total_network")
N=200
moving_av = baits_sort.rolling(N).mean()
moving_std = baits_sort.rolling(N).std()

fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.set_xlabel('time(t)')
ax.set_ylabel('speed($\mu m. h^{-1}$)')
xlim = 40
# ax.set_xlim(0,60)
# ax.set_ylim(0,400)
# slope, intercept, r_value, p_value, std_err = stats.linregress(densities_sort,np.abs(curvatures_sort))
x= range(-xlim,xlim)
for plate in set(treatment_25['plate'].values):
    select = treatment_25.loc[treatment_25['plate']==plate]
    ax.scatter(select["t"],select['total_network'],label=f'{plate}')
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [46]:
for plate in set(treatment_25['plate'].values):
    select = treatment_25.loc[treatment_25['plate'] == plate]
    break

In [47]:
select

,plate,inst,treatment,angle,curvature,density,growth,speed,straightness,t,hyph,total_network
2900,102,"(40, 262, 287)",25*,5.015835,0.004589,1.281159,1093.099538,274.418294,0.996673,0.000000,68,3.359985e+04
2901,102,"(40, 262, 287)",25*,-9.041261,-0.008538,0.681159,1059.002674,264.750669,0.992007,3.983333,68,3.777911e+04
2902,102,"(40, 262, 287)",25*,88.501522,0.170012,0.648551,520.560350,130.140088,0.851340,7.983333,68,4.403740e+04
2903,102,"(40, 262, 287)",25*,5.685028,0.004396,1.044928,1293.328033,323.332008,0.996512,91.483333,71,1.168615e+06
2904,102,"(40, 262, 287)",25*,19.549688,0.016276,0.635507,1201.144760,300.286190,0.989235,95.483333,71,1.301554e+06
...,...,...,...,...,...,...,...,...,...,...,...,...
3226,102,"(40, 262, 287)",25*,-0.463923,-0.000460,1.078986,1008.838430,252.209608,0.991182,95.483333,14048,1.301554e+06
3227,102,"(40, 262, 287)",25*,-16.888398,-0.023034,1.553623,733.193080,183.298270,0.997701,95.483333,14096,1.301554e+06
3228,102,"(40, 262, 287)",25*,-9.829751,-0.014136,1.974638,695.387948,173.846987,0.988627,95.483333,14534,1.301554e+06
3229,102,"(40, 262, 287)",25*,1.551381,0.002390,1.977536,649.115712,162.278928,0.990175,95.483333,14548,1.301554e+06
